## Starting Model

In [35]:
import boto3
import io
from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont


In [36]:
s3 = boto3.resource('s3')

In [37]:


def start_model(project_arn, model_arn, version_name, min_inference_units):

    client=boto3.client('rekognition')

    try:
        # Start the model
        print('Starting model: ' + model_arn)
        response=client.start_project_version(ProjectVersionArn=model_arn, MinInferenceUnits=min_inference_units)
        # Wait for the model to be in the running state
        project_version_running_waiter = client.get_waiter('project_version_running')
        project_version_running_waiter.wait(ProjectArn=project_arn, VersionNames=[version_name])

        #Get the running status
        describe_response=client.describe_project_versions(ProjectArn=project_arn,
            VersionNames=[version_name])
        for model in describe_response['ProjectVersionDescriptions']:
            print("Status: " + model['Status'])
            print("Message: " + model['StatusMessage']) 
    except Exception as e:
        print(e)
        
    print('Done...')
    
def main():
    project_arn='arn:aws:rekognition:us-east-1:735074111034:project/PropertyBot-v3-room-rekognition/1630820983471'
    model_arn='arn:aws:rekognition:us-east-1:735074111034:project/PropertyBot-v3-room-rekognition/version/PropertyBot-v3-room-rekognition.2021-09-04T22.57.53/1630821474130'
    min_inference_units=1 
    version_name='PropertyBot-v3-room-rekognition.2021-09-04T22.57.53'
    start_model(project_arn, model_arn, version_name, min_inference_units)

In [38]:
main()

Starting model: arn:aws:rekognition:us-east-1:735074111034:project/PropertyBot-v3-room-rekognition/version/PropertyBot-v3-room-rekognition.2021-09-04T22.57.53/1630821474130
An error occurred (ResourceInUseException) when calling the StartProjectVersion operation: ProjectVersion arn:aws:rekognition:us-east-1:735074111034:project/PropertyBot-v3-room-rekognition/version/PropertyBot-v3-room-rekognition.2021-09-04T22.57.53/1630821474130 is RUNNING; cannot be started
Done...


## Analyzing an Image

In [39]:
def show_custom_labels(model,bucket,photo, min_confidence):
    client=boto3.client('rekognition')

    #Call DetectCustomLabels
    response = client.detect_custom_labels(Image={'S3Object': {'Bucket': bucket, 'Name': photo}},
        MinConfidence=min_confidence,
        ProjectVersionArn=model)

    # For object detection use case, uncomment below code to display image.
    # display_image(bucket,photo,response)

    return response['CustomLabels']

In [40]:


def label_and_save_in_bucket(prefix, images_left):
    room = ''
    bucket='propertybot-v3'

    model='arn:aws:rekognition:us-east-1:735074111034:project/PropertyBot-v3-room-rekognition/version/PropertyBot-v3-room-rekognition.2021-09-04T22.57.53/1630821474130'
    min_confidence=30

    #labeling image
    label_count=show_custom_labels(model,bucket,  prefix, min_confidence)
    try:
        # print("LABEL COUNT: {0}".format(label_count))
        room = label_count[0]['Name'].lower()
        print("INFO: label from model is {0}".format(room))


        # aggregating labels
        if room == "back yard" or room =="front yard":
            room = "outdoor"
        elif room == "kitchen":
            room = "kitchen"
        elif room == "garage":
            room = "garage"
        elif room == "bathroom":
            room = "bathroom"
        elif room == "bedroom":
            room  = "other-rooms"
        elif room == "living room":
            room = "other-rooms"
        else:
            print("INFO: identified a {0}".format(room))
            return images_left

        # print("INFO: identified a {0}".format(room))

        if(images_left[room] == 0):
            return images_left

        images_left[room] = images_left[room] - 1

        s3 = boto3.resource('s3')
        copy_source = {
              'Bucket': bucket,
              'Key': prefix
            }
        sink_bucket = s3.Bucket("training-images-{0}".format(room))
        sink_bucket.copy(copy_source, prefix.split("/")[-1])
        # print("INFO: saved images in s3 bucket for {0}".format(room))
        return images_left
    except Exception as e:
        
        print(e)
 



In [41]:
print(label_and_save_in_bucket(prefix="data/raw/images/Cleveland/M3142060591_109.png", images_left={'other-rooms': 500, 'outdoor': 500, 'kitchen': 500, 'bathroom': 500, 'other-rooms': 500, 'garage': 500}))

INFO: label from model is kitchen
{'other-rooms': 500, 'outdoor': 500, 'kitchen': 499, 'bathroom': 500, 'garage': 500}


In [45]:
bucket = s3.Bucket('propertybot-v3')
counter = 0
image_dict = {'other-rooms': 500, 'outdoor': 500, 'kitchen': 500, 'bathroom': 500, 'other-rooms': 500, 'garage': 500}
for object_summary in bucket.objects.filter(Prefix="data/raw/images/Cleveland/"):
    counter = counter +1 
    if(counter < 110):
        continue
    image_dict = label_and_save_in_bucket(prefix=object_summary.key, images_left=image_dict) or image_dict
    print(image_dict)
    print(counter)
    if not any(image_dict.values()):
        break

INFO: label from model is storage/closet
INFO: identified a storage/closet
{'other-rooms': 500, 'outdoor': 500, 'kitchen': 500, 'bathroom': 500, 'garage': 500}
110
INFO: label from model is basement
INFO: identified a basement
{'other-rooms': 500, 'outdoor': 500, 'kitchen': 500, 'bathroom': 500, 'garage': 500}
111
INFO: label from model is back yard
{'other-rooms': 500, 'outdoor': 499, 'kitchen': 500, 'bathroom': 500, 'garage': 500}
112
INFO: label from model is back yard
{'other-rooms': 500, 'outdoor': 498, 'kitchen': 500, 'bathroom': 500, 'garage': 500}
113
INFO: label from model is front yard
{'other-rooms': 500, 'outdoor': 497, 'kitchen': 500, 'bathroom': 500, 'garage': 500}
114
INFO: label from model is living room
{'other-rooms': 499, 'outdoor': 497, 'kitchen': 500, 'bathroom': 500, 'garage': 500}
115
INFO: label from model is living room
{'other-rooms': 498, 'outdoor': 497, 'kitchen': 500, 'bathroom': 500, 'garage': 500}
116
INFO: label from model is living room
{'other-rooms': 